# VariantFormer: Variant-to-Expression Prediction

## Predicting the Functional Impact of Genetic Variants on Gene Expression

### Executive Summary

VariantFormer is a state-of-the-art deep learning framework that predicts how genetic variants affect gene expression across diverse tissues and populations. This notebook demonstrates the core functionality of the VariantFormer system, showcasing how single nucleotide variants (SNVs) can be analyzed to understand their potential impact on cellular function.

### Innovation

VariantFormer addresses several critical limitations of existing approaches:

1. **Multi-tissue Analysis**: Predicts variant effects across 62 tissues and cell types
2. **Population-aware**: Considers genetic background from major world populations
3. **Mechanistic Understanding**: Models both cis-regulatory elements (CREs) and gene sequences
---


## Methodology Overview

VariantFormer employs a sophisticated **transformer-based architecture** that integrates DNA sequences around genes and regulatory elements.

### Scoring Strategies

#### 1. Expression Prediction Scores
- **Direct Output**: Predicted gene expression values
- **Comparison**: Reference (wild-type) vs. heterozygous vs. homozygous variant genotypes
- **Tissue-specific**: Separate predictions for each tissue type

#### 2. Log2 Fold Change (log2fc) Regulation Scores as eQTLs
- **Population Comparison**: Analyzes expression differences across populations
- **Reference Comparison**: Compares variant expression to reference genome expression  
- **Directional Interpretation**: 
  - **Positive log2fc (>0)**: Gene upregulation - variant increases expression
  - **Negative log2fc (<0)**: Gene downregulation - variant decreases expression  
  - **log2fc ≈ 0**: Minimal effect - variant has little impact
- **Population Weighting**: Aggregates scores across populations using allele frequency weights
- **Magnitude**: Larger absolute values indicate stronger regulatory effects

## Initialize VariantFormer

The `v4_ag` model supports all genes (protein-coding and non-coding). Use `v4_pcg` for protein-coding genes only.


In [ ]:
# Essential imports
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

import ipynbname

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from processors.variantprocessor import VariantProcessor
REPO_PATH = ipynbname.path().parent.parent


In [ ]:
# Initialize VariantFormer
print("🚀 Initializing VariantFormer Variant Processor...")
model_class = 'v4_ag' # model class can be 'v4_ag', 'v4_pcg'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'v4_pcg' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)

print("✅ System initialized!")
print(f"🌍 Populations: {', '.join(vep.populations)}")
print(f"🧬 Tissues: {len(vep.tissue_vocab)} available")

## Define Variant for Analysis

Specify variant coordinates, alleles, target tissue, and gene ID.


In [ ]:
# Define variants for analysis
print("🧬 Creating multi-variant dataset...")

multi_variants = {
    "chr": ["chr13"],
    "pos": [
        113978728,
    ],
    "ref": [
        "A",
    ],
    "alt": ["G"],
    "tissue": ["whole blood"],
    "gene_id": ["ENSG00000185989.10"],
}

variant_df = pd.DataFrame(multi_variants)
print(variant_df.to_string(index=False))


## Run Predictions

Generate expression predictions across populations and zygosity states.


In [ ]:
# Run variant predictions
print("🔬 Running VariantFormer variant analysis...")
output_dir = "/tmp/variant_multi_output"
# Predict expression effects for all variants
raw_predictions = vep.predict(var_df=variant_df, output_dir=output_dir)

In [ ]:
raw_predictions.head()

**Prediction Output Schema**
- `chrom`, `pos`, `ref`, `alt`: Variant coordinates and alleles
- `genes`: Ensembl Gene ID
- `tissue`: Tissue name
- `population`: Population code (REF_HG38, EAS, EUR, AFR, SAS, AMR)
- `sample_name`: 1000 Genomes sample identifier
- `zygosity`: 0 (homozygous ref), 1 (heterozygous), 2 (homozygous alt)
- `gene_exp`: Predicted gene expression level for the gene in the specified tissue and sample
- `variant_type`: Gene Overlap / CRE Overlap
- `gene_emb`: Embedding of the gene conditioned on tissue and regulatory context from the last transformer layer
- `gene_token_embedding`:  Embedding of the token that overlaps with the variant position from the last transformer layer
- `cre_token_embedding`: Embeddings of regulatory elements overlapping with the variant position from the last transformer layer


## Format Scores and Calculate eQTL Effects

Transform raw predictions into structured format and compute log2 fold changes across populations.


In [ ]:
# Format and score the predictions
print("📊 Formatting and scoring predictions...")

# Step 1: Format scores
formatted_scores = vep.format_scores(raw_predictions)


# Step 2: Calculate eQTL scores (log2fc-based)
final_results = vep.eqtl_scores(formatted_scores)
print(f"✅ Calculated regulation scores for {len(final_results)} variants")
final_results.head()

**Population-specific log2 fold changes** (homozygous alt vs. ref):
- `VF-{POP}-2-exp-log2fc`: Log2 fold change in expression for the homozygous alternative allele in the background genome compared to reference' (Background POP: AFR, EUR, EAS, SAS, AMR)
- `VF-REF_HG38-2-exp-log2fc`: Expression change in reference genome
- `VF-agg-log2fc-weighted`: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref


## Score Interpretation

- **log2fc > 0**: Upregulation (≥1.0: strong, ≥0.5: moderate)
- **log2fc ≈ 0**: Neutral effect
- **log2fc < 0**: Downregulation (≤-0.5: moderate, ≤-1.0: strong)


## Key Findings & Applications

### Variant-level Insights
VariantFormer enables simultaneous analysis of variant sets to:
- **Prioritize variants** by regulatory impact
- **Compare effects** across different genomic loci  
- **Identify tissue-specific** regulatory variants
- **Population-stratified** variant interpretation

### Clinical Applications
- Focus on functionally relevant variants  
- Population-aware variant interpretation  
- Identify expression-modulating variants  
- Prioritize causal variants from association studies  



---

# VCF-Based Analysis

VariantFormer can analyze variants insilico in individual genotype context using VCF files for personalized genomic insights.


In [ ]:
# Example: VCF-based Analysis
print("🧬 VCF Integration Example")
print("=" * 40)

# VCF files contain individual genotype information
# This enables more precise analysis based on actual sample data

In [ ]:
# For VCF analysis, we can reuse the existing VariantProcessor
print("🔄 Using existing VariantFormer system for VCF analysis...")
print("📁 VCF files enable individual-specific genomic analysis")

## Define VCF Parameters

Specify VCF file path, sample name, and variant coordinates for individual-specific analysis.


In [ ]:
model_class = 'v4_ag' # model class can be 'v4_ag', 'v4_pcg'. AG model is all-genes model trained on both protein-coding and non-coding genes.
# model_class = 'v4_pcg' # Uncomment to use the PCG model
vep = VariantProcessor(model_class=model_class)

In [ ]:
# Example VCF integration parameters
vcf_path = os.path.join(str(Path.cwd().parent),"_artifacts/HG00096.vcf.gz") # Change path as needed
sample_name = "HG00096"  # European sample from 1000 Genomes Project


# Same variant for insilico analysis. Alt allele will be placed in the sample genotype context at that specific position. 
# Ref is from reference genome.
vcf_variant_data = {
    "chr": ["chr13"],
    "pos": [113978728],
    "ref": ["A"],
    "alt": ["G"],
    "tissue": ["whole blood"],
    "gene_id": ["ENSG00000185989.10"],
}

vcf_variant_df = pd.DataFrame(vcf_variant_data)
print("\n🧬 Analyzing the same variant with individual genotype data:")
print(vcf_variant_df.to_string(index=False))

## Run VCF-Based Predictions

Generate predictions with individual genotype context from the VCF file.


In [ ]:
# Run VariantFormer with VCF integration
print("\n🔬 Running VariantFormer analysis with VCF integration...")
print("⏳ Processing individual genotype data...")

vcf_predictions = vep.predict(
    var_df=vcf_variant_df,
    output_dir="/tmp/variant_output_vcf",
    vcf_path=vcf_path,
    sample_name=sample_name,
)

print("✅ VCF-based predictions completed!")


## VCF Output Format

**Expression columns:**
- `REF_HG38-{0,1,2}-exp`: Expression in reference genome (homozygous ref, het, homozygous alt)
- `SAMPLE-0-exp`: Predicted expression for the individual without perturbation at that locus')
- `SAMPLE-1-exp`: Predicted expression for the individual with heterozygous alternate at that locus')
- `SAMPLE-2-exp`: Predicted expression for the individual with homozygous alternate at that locus')

In [ ]:
# Format scores for VCF-based predictions
vcf_formatted_scores = vep.format_scores(vcf_predictions)
vcf_formatted_scores.head()

In [ ]:
# Apply eQTL scoring to VCF results
vcf_final_results = vep.eqtl_scores(vcf_formatted_scores)

print("✅ VCF-based eQTL scoring completed!")

vcf_final_results.head()

**column interpretations**

- `VF-REF_HG38-2-exp-log2fc`: Log2 fold change in expression for the homozygous alternative allele in the HG38 reference genome compared to reference homozygous ref
- `VF-SAMPLE-2-exp-log2fc`: Log2 fold change in expression for the homozygous alternative allele in the individual sample compared to reference
